<a href="https://colab.research.google.com/github/budennovsk/Pandas/blob/master/beauty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall pyspark -y  # Удалите текущую
!pip install pyspark==3.2.1  # Установите 3.2.1 (проверьте на compatibility с replay)

Found existing installation: pyspark 3.2.1
Uninstalling pyspark-3.2.1:
  Successfully uninstalled pyspark-3.2.1
  Using cached pyspark-3.2.1-py2.py3-none-any.whl
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 1.0.2 requires pyspark[connect]~=4.0.0, but you have pyspark 3.2.1 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import pickle
import sys

sys.path.append('../..')

import numpy as np
import polars as pl
from dotenv import load_dotenv
from replay.preprocessing.filters import MinCountFilter
from sentence_transformers import SentenceTransformer

In [3]:
load_dotenv()

False

In [4]:
import polars as pl

# path_beauty = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/Amazon_Beauty_Recommendation.csv'
path_parquet = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/Amazon_Beauty_Recommendation.parquet'
# df = pl.read_csv(path_beauty)
# df.write_parquet('/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/Amazon_Beauty_Recommendation.parquet')
# print("Готово! Parquet в /content/Amazon_Beauty_Recommendation.parquet")

interactions = pl.read_parquet(path_parquet)

print("Shape:", interactions.shape)
print("Колонки:", interactions.columns)
print("Пример данных:")
print(interactions.head(5))

Shape: (1348246, 6)
Колонки: ['UserId', 'ProductId', 'ProductType', 'Rating', 'Timestamp', 'URL']
Пример данных:
shape: (5, 6)
┌────────────────┬────────────┬──────────────────┬────────┬────────────┬───────────────────────────┐
│ UserId         ┆ ProductId  ┆ ProductType      ┆ Rating ┆ Timestamp  ┆ URL                       │
│ ---            ┆ ---        ┆ ---              ┆ ---    ┆ ---        ┆ ---                       │
│ str            ┆ str        ┆ str              ┆ i64    ┆ i64        ┆ str                       │
╞════════════════╪════════════╪══════════════════╪════════╪════════════╪═══════════════════════════╡
│ A3NHUQ33CFH3VM ┆ B00LLPT4HI ┆ Eyeliner & Kajal ┆ 5      ┆ 1405814400 ┆ https://www.amazon.in/May │
│                ┆            ┆                  ┆        ┆            ┆ belli…                    │
│ A1TIRNQ7O4REOH ┆ B00LLPT4HI ┆ Eyeliner & Kajal ┆ 4      ┆ 1405987200 ┆ https://www.amazon.in/May │
│                ┆            ┆                  ┆        ┆      

In [5]:
# Select columns
interactions = interactions.select(
    pl.col('UserId').alias('user_id'),
    pl.col('ProductId').alias('item_id'),
    pl.col('Rating').alias('rating'),
    pl.col('Timestamp').alias('timestamp'),
)
interactions.head(1)

user_id,item_id,rating,timestamp
str,str,i64,i64
"""A3NHUQ33CFH3VM""","""B00LLPT4HI""",5,1405814400


In [6]:
# Фильтрация по UserId
filtered_df =interactions.filter(interactions['user_id'] == "A1YJEY40YUW4SE")

print("Фильтрованные данные:")
print(filtered_df)

Фильтрованные данные:
shape: (4, 4)
┌────────────────┬────────────┬────────┬────────────┐
│ user_id        ┆ item_id    ┆ rating ┆ timestamp  │
│ ---            ┆ ---        ┆ ---    ┆ ---        │
│ str            ┆ str        ┆ i64    ┆ i64        │
╞════════════════╪════════════╪════════╪════════════╡
│ A1YJEY40YUW4SE ┆ B004ZT0SSG ┆ 5      ┆ 1318896000 │
│ A1YJEY40YUW4SE ┆ B004756YJA ┆ 5      ┆ 1318896000 │
│ A1YJEY40YUW4SE ┆ B002WLWX82 ┆ 1      ┆ 1391040000 │
│ A1YJEY40YUW4SE ┆ B0020YLEYK ┆ 5      ┆ 1328140800 │
└────────────────┴────────────┴────────┴────────────┘


In [7]:
# Keep positive interactions only
MIN_RATING = 3

interactions = interactions.filter(pl.col('rating') > MIN_RATING).drop('rating')
len(interactions)

1037636

In [8]:
interactions.select(pl.col('user_id', 'item_id').n_unique())

user_id,item_id
u32,u32
709194,23837


In [9]:
# Split data into train/val and test subsets
TIME_THRESHOLD_QUANTILE = 0.9
TIME_THRESHOLD = interactions.get_column('timestamp').quantile(TIME_THRESHOLD_QUANTILE)

condition = pl.col('timestamp') > TIME_THRESHOLD

train_val_interactions = interactions.filter(~condition)
test_interactions = interactions.filter(condition)

len(train_val_interactions), len(test_interactions)

(935085, 102551)

In [10]:
# Apply N-core filtering to train/val data
N = 3
shape = None

while shape != train_val_interactions.shape:
    shape = train_val_interactions.shape

    train_val_interactions = MinCountFilter(N, groupby_column='user_id').transform(
        train_val_interactions
    )
    train_val_interactions = MinCountFilter(N, groupby_column='item_id').transform(
        train_val_interactions
    )

len(train_val_interactions)

249136

In [11]:
# Assign tail interactions back to test users
test_users = test_interactions.get_column('user_id').unique()
test_interactions = pl.concat(
    [test_interactions, train_val_interactions.filter(pl.col('user_id').is_in(test_users))]
)
len(test_interactions)

/tmp/ipython-input-2450227530.py:4: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  [test_interactions, train_val_interactions.filter(pl.col('user_id').is_in(test_users))]


141321

In [12]:
# Split data into train and validation subsets
TRAIN_USERS_FRACTION = 0.9
TRAIN_USERS = (
    train_val_interactions.get_column('user_id')
    .unique()
    .sort()
    .sample(fraction=TRAIN_USERS_FRACTION, seed=42)
)

condition = pl.col('user_id').is_in(TRAIN_USERS)

train_interactions = train_val_interactions.filter(condition)
val_interactions = train_val_interactions.filter(~condition)

len(train_interactions), len(val_interactions)

/tmp/ipython-input-1666146712.py:12: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  train_interactions = train_val_interactions.filter(condition)
/tmp/ipython-input-1666146712.py:13: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  val_interactions = train_val_interactions.filter(~condition)


(224239, 24897)

In [13]:
# Encode warm item IDs
WARM_ITEMS = train_interactions.get_column('item_id').unique().sort()
print(f'{len(WARM_ITEMS) = }')

# Keep zero for padding
item2index_warm = {item: index + 1 for index, item in enumerate(WARM_ITEMS)}

train_interactions = train_interactions.with_columns(
    pl.col('item_id').replace_strict(item2index_warm)
)

# Filter out cold items from validation subset
val_interactions = val_interactions.filter(pl.col('item_id').is_in(WARM_ITEMS))
val_interactions = val_interactions.with_columns(pl.col('item_id').replace_strict(item2index_warm))

len(WARM_ITEMS) = 18534


/tmp/ipython-input-502203731.py:13: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  val_interactions = val_interactions.filter(pl.col('item_id').is_in(WARM_ITEMS))


In [14]:
# Encode cold item IDs
test_interactions = test_interactions.with_columns(
    ~pl.col('item_id').is_in(WARM_ITEMS).alias('is_cold')
)

COLD_ITEMS = test_interactions.filter(pl.col('is_cold')).get_column('item_id').unique().sort()
print(f'{len(COLD_ITEMS) = }')

bias = max(item2index_warm.values()) + 1
item2index_cold = {item: index + bias for index, item in enumerate(COLD_ITEMS)}

test_interactions = test_interactions.with_columns(
    pl.col('item_id').replace_strict({**item2index_warm, **item2index_cold})
)

len(COLD_ITEMS) = 3617


/tmp/ipython-input-2285056509.py:2: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  test_interactions = test_interactions.with_columns(


In [28]:
# Separate ground truth from test interactions
test_interactions = test_interactions.with_columns(
    pl.col('user_id')
    .cum_count(reverse=True)
    .over('user_id', order_by='timestamp')
    .alias('position')
)

condition = pl.col('position') == 1

ground_truth = test_interactions.filter(condition)
test_interactions = test_interactions.filter(~condition)

In [30]:
len(train_interactions), len(val_interactions), len(test_interactions), len(ground_truth)

(224239, 24886, 62419, 78902)

In [33]:
# Apply some basic checks
assert (train_interactions.get_column('timestamp') <= TIME_THRESHOLD).all()
assert (val_interactions.get_column('timestamp') <= TIME_THRESHOLD).all()
assert (ground_truth.get_column('timestamp') > TIME_THRESHOLD).all()

In [103]:
import polars as pl

cold_user_ids = ground_truth.filter(pl.col('is_cold') == True).get_column('item_id').unique().to_list()



In [105]:
filtered_val =test_interactions.filter(pl.col('item_id').is_in(cold_user_ids)).unique()
filtered_val

user_id,item_id,timestamp,is_cold,position
str,i64,i64,bool,u32
"""A2YL6HWOTVSJ8B""",22130,1405382400,true,2
"""AFJQFSIUWBV9U""",21933,1400198400,true,3
"""AWV5D40RG0SVB""",22121,1404259200,true,2
"""A3UCSKE14KSNK7""",21827,1405382400,true,2
"""A2SK56K7VLRF8W""",22133,1404950400,true,2
…,…,…,…,…
"""A1YBCADD0129DR""",21948,1404691200,true,2
"""A1HL46MFQKYWAU""",22112,1402704000,true,5
"""ANS2X1R0B7DHD""",21936,1402876800,true,2


In [106]:
# Save processed data
OUTPUT_DIR = '/content/drive/MyDrive/Colab Notebooks/Симбирсофт/recsys/data-20260210T140001Z-1-001/формированаядата'

if not os.path.exists(os.path.join(OUTPUT_DIR, 'processed')):
    os.makedirs(os.path.join(OUTPUT_DIR, 'processed'))

train_interactions.write_parquet(os.path.join(OUTPUT_DIR, 'processed/train_interactions.parquet'))
val_interactions.write_parquet(os.path.join(OUTPUT_DIR, 'processed/val_interactions.parquet'))
test_interactions.write_parquet(os.path.join(OUTPUT_DIR, 'processed/test_interactions.parquet'))
ground_truth.write_parquet(os.path.join(OUTPUT_DIR, 'processed/ground_truth.parquet'))

with open(os.path.join(OUTPUT_DIR, 'processed/item2index_warm.pkl'), mode='wb') as file:
    pickle.dump(item2index_warm, file)

with open(os.path.join(OUTPUT_DIR, 'processed/item2index_cold.pkl'), mode='wb') as file:
    pickle.dump(item2index_cold, file)

In [ ]:
metadata = pl.read_parquet(os.path.join(BEAUTY_DATA_DIR, 'meta_Beauty.parquet'))
print(f'{metadata.shape = }')
metadata.head(1)

metadata.shape = (259204, 9)


asin,description,title,imUrl,salesRank,categories,price,related,brand
str,str,str,str,struct[29],list[list[str]],f64,struct[4],str
"""0205616461""","""As we age, our once youthful, …","""Bio-Active Anti-Aging Serum (F…","""http://ecx.images-amazon.com/i…","{null,null,null,null,null,null,null,null,null,null,null,461765,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null}","[[""Beauty"", ""Skin Care"", … ""Creams & Moisturizers""]]",null,"{null,null,null,null}",null


In [ ]:
# Process metadata
metadata = (
    metadata.rename({'asin': 'item_id'})
    .filter(pl.col('item_id').is_in(pl.concat((WARM_ITEMS, COLD_ITEMS))))
    .with_columns(pl.col('item_id').replace_strict({**item2index_warm, **item2index_cold}))
    .sort('item_id')
)
print(f'{metadata.shape = }')
metadata.head(1)

metadata.shape = (11733, 9)


item_id,description,title,imUrl,salesRank,categories,price,related,brand
i64,str,str,str,struct[29],list[list[str]],f64,struct[4],str
1,"""Prada Candy By Prada Eau De Pa…","""Prada Candy By Prada Eau De Pa…","""http://ecx.images-amazon.com/i…","{null,null,null,78916,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null}","[[""Beauty"", ""Fragrance"", … ""Eau de Parfum""]]",65.86,"{[""B006C5OHSI"", ""B006P14842"", … ""9788072208""],[""B0072CSVB4"", ""B005YWBOHW"", … ""B000C1Z3LS""],[""B006C5OHSI"", ""B006P14842""],null}","""Prada"""


In [ ]:
assert len(metadata) == len(item2index_warm) + len(item2index_cold)

In [ ]:
CHECKPOINT = 'intfloat/e5-base-v2'

model = SentenceTransformer(CHECKPOINT)

item_embeddings = model.encode(
    metadata.get_column('title').to_list(),
    batch_size=512,
    show_progress_bar=True,
    normalize_embeddings=False,
)
item_embeddings.shape

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

(11733, 768)

In [ ]:
warm_item_embeddings = item_embeddings[: len(WARM_ITEMS)]
cold_item_embeddings = item_embeddings[len(WARM_ITEMS) :]

In [ ]:
# Save item embeddings
if not os.path.exists(os.path.join(OUTPUT_DIR, 'item_embeddings')):
    os.makedirs(os.path.join(OUTPUT_DIR, 'item_embeddings'))

np.save(os.path.join(OUTPUT_DIR, 'item_embeddings/embeddings_warm.npy'), warm_item_embeddings)
np.save(os.path.join(OUTPUT_DIR, 'item_embeddings/embeddings_cold.npy'), cold_item_embeddings)